In [1]:
import xarray as xr
import numpy as np
import os
import sys
sys.path.append("../..")
from surgeNN.evaluation import add_error_metrics_to_prediction_ds
import gcsfs
fs = gcsfs.GCSFileSystem()

Script intended to correct original output for alignment with GTSM simulations with stop slightly earlier than the end of 2017:

In [6]:
#configure the script
tgs        = ['stavanger-svg-nor-nhs.csv','wick-wic-gbr-bodc.csv','esbjerg-esb-dnk-dmi.csv','immingham-imm-gbr-bodc.csv','den_helder-denhdr-nld-rws.csv', 'fishguard-fis-gbr-bodc.csv',  'brest-822a-fra-uhslc.csv', 'vigo-vigo-esp-ieo.csv',  'alicante_i_outer_harbour-alio-esp-da_mm.csv']
tgnames = ['Stavanger (NOR)','Wick (UK)', 'Esbjerg (DK)','Immingham (UK)','Den Helder (NL)','Fishguard (UK)','Brest (FR)','Vigo (PT)', 'Alicante (SP)']

qnts = np.array([.95,.98,.99,.995]) #quantiles, don't touch

Correct performance output datasets that were based on 10 more timesteps at the end of 2017 than GTSM:
**be careful as this overwrites results!**

#MLR
path = '/home/jovyan/test_surge_models/results/mlr_4x4/performance'
files = os.listdir(path)
for f in files:
    if f.startswith('.') == False:
        print('current_file:' +f)
        ds = xr.open_dataset(os.path.join(path,f)).load()
        
        #1
        ds = ds.drop_vars(['true_pos_self','true_neg_self','false_pos_self','false_neg_self',
                            'precision_self','recall_self','f1_self'],errors='ignore')
        
        #2
        if 'max_timesteps_between_extremes' in ds:
            print('"max_timesteps_between_extremes" is already a dimension, file already updated? Skipping.')
        else:   
            for metric in ['r_extremes','rmse_extremes','true_pos','false_neg','false_pos','true_neg','precision','recall','f1']:
                ds[metric] = ds[metric].expand_dims(dim='max_timesteps_between_extremes') #add additional dimension for max time distance used between 

            ds['max_timesteps_between_extremes'] = [-1]

            ds_max_dist_3 = add_error_metrics_to_prediction_ds(ds.copy(deep=True),qnts,3)
            ds_new = xr.merge((ds,ds_max_dist_3))

            ds.close()
            ds_new.to_netcdf(os.path.join(path,f.replace('.nc','_v2.nc')),mode='w') #store new dataset
            del ds,ds_new
            
            os.remove(os.path.join(path,f)) #delete old file
            os.rename(os.path.join(path,f.replace('.nc','_v2.nc')),os.path.join(path,f))#rename new file

In [3]:
#LSTMs
path = 'gs://leap-persistent/timh37/surgeNN_output/nns/performance_modified_v2/lstm'
files = fs.ls(path)
for f in files:
    if f.startswith('.') == False:
        print('current_file:' +f)
        ds = xr.open_dataset('gs://'+f,engine='zarr')
        
        if ds.time[-1]==np.datetime64('2017-12-31T21:00'):
            ds = ds.isel(time=np.arange(len(ds.time)-10)) #adjust to match GTSM coverage
        
        try: 
            ds = ds.isel(max_timesteps_between_extremes=-1,drop=True)
        except:
            pass
        
        ds_new = add_error_metrics_to_prediction_ds(ds.copy(deep=True),qnts,3)
        
        ds.close()
        
        for k in ['p','tg','split']:
            ds_new[k] = ds_new[k].astype('str')
        ds_new.to_zarr('gs://'+f.replace('performance_modified','performance_modified_v2'),mode='w',zarr_version=2)
        

current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/lstm/lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it0.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/lstm/lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it1.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/lstm/lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it2.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/lstm/lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it3.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/lstm/lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it4.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/lstm/lstm_3h_brest-822a-fra-uhslc_mse_hp1_it0.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/lstm/lstm_3h_brest-822a-fra-uhslc_mse_hp1_it1.zarr
current_file:lea

In [7]:
#ConvLSTMs
path = 'gs://leap-persistent/timh37/surgeNN_output/nns/performance_modified/convlstm'
files = fs.ls(path)
for f in files:    
    if f.startswith('.') == False:
        print('current_file:' +f)
        ds = xr.open_dataset('gs://'+f,engine='zarr')
        
        if ds.time[-1]==np.datetime64('2017-12-31T21:00'):
            ds = ds.isel(time=np.arange(len(ds.time)-10)) #adjust to match GTSM coverage
        
        try: 
            ds = ds.isel(max_timesteps_between_extremes=-1,drop=True)
        except:
            pass
        
        ds_new = add_error_metrics_to_prediction_ds(ds.copy(deep=True),qnts,3)
        
        for k in ['p','tg','split']:
            ds_new[k] = ds_new[k].astype('str')
        
        ds_new.to_zarr('gs://'+f.replace('performance_modified','performance_modified_v2'),mode='w',zarr_version=2)
        

current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/convlstm/convlstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it0.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/convlstm/convlstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it1.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/convlstm/convlstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it2.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/convlstm/convlstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it3.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/convlstm/convlstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it4.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/convlstm/convlstm_3h_brest-822a-fra-uhslc_mse_hp1_it0.zarr
current_file:leap-persistent/timh37/surgeNN_output/nns/performance_modified/convlstm/convlstm_3